# M.SMART

Задача: разработка модели предсказания оценки по тексту отзыва.

Данные: отзывы о товарах с сайта mvideo.ru, оценки, которые поставлены авторами отзыва, категория и брэнд товара. 

Цель: Предсказание общей оценки товара

Формат предоставления решения: Ссылка на репозиторий на GitHub с исходным кодом программы, решающей поставленную задачу. На вход программа должна принимать данные в исходном виде (feedback), разбивать на train и test, обучать на train модель и демонстрировать ее качество на test. Репозиторий должен содержать проект для среды разработки и инструкцию по сборке и запуску. Рекомендуем использовать Jupyter Notebook, но окончательный выбор инструментария остаётся за вами.

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

**Загрузка данных**

In [2]:
reviews = pd.read_csv('X_train.csv')
reviews.head(5)

,sku,categoryLevel1Id,categoryLevel2Id,brandId,property,userName,reting,date,comment,commentNegative,commentPositive
0,20005023,401,4010201,826,"[{34: 'f982777489055c6563d68c005fd24aad'}, {36...",b2898a81b45310b30beb8fc0c0a9ce1e,2.0,2013-06-28,"2,5 года работала и все...устала! Лампочка гор...",NaN,NaN
1,20020647,403,4030101,1425,"[{34: '9ce895413ebdf6b6dcb69b07dc782591'}, {36...",538c73d64461e13907bb95c51c38bfbc,2.0,2010-07-04,Через 2 месяца после истечении гарантийного ср...,NaN,NaN
2,20020701,401,4010401,124,"[{34: '9ce895413ebdf6b6dcb69b07dc782591'}, {36...",ddca2d0101513a6209db7868eed8be05,4.0,2010-05-27,пользуюсь уже три недели. нареканий ни каких н...,NaN,NaN
3,30012256,203,2030301,93,"[{34: '9ce895413ebdf6b6dcb69b07dc782591'}, {36...",289c20015b3713a82ba5ddf774d996f7,5.0,2016-10-11,Ребят этот системный блок подойдёт для игры кс...,NaN,NaN
4,30011341,205,2050201,656,"[{34: '9ce895413ebdf6b6dcb69b07dc782591'}, {36...",5576f82d149d4f688644fef2322c63ef,5.0,2010-02-26,"я считаю, что яри замечательный телефон! Прият...",NaN,NaN


**Функция для балансировки выборки**

In [3]:
def balance_classes(xs, ys):
    freqs = Counter(ys)

    # Возьмем из каждого класса одинаковое количество отзывов
    max_allowable = freqs.most_common()[-1][1]
    num_added = {clss: 0 for clss in freqs.keys()}
    new_ys = []
    new_xs = []
    for i, y in enumerate(ys):
        if num_added[y] < max_allowable:
            new_ys.append(y)
            new_xs.append(xs[i])
            num_added[y] += 1
    return new_xs, new_ys

**Функция для обучения классификатора**

In [4]:
def rate_reviews(reviews):
    
    # Возьмем нужные столбцы
    texts = reviews.comment
    rating = reviews.reting.map(lambda x: np.round(x))
    
    # Балансируем выборку
    balanced_x, balanced_y = balance_classes(texts, rating)
    
    # Разбиваем на n-граммы и векторизируем комментарии
    vectorizer = TfidfVectorizer(ngram_range=(1,3))
    vectors = vectorizer.fit_transform(balanced_x)
    
    # Разобъем на тренировочную и тестирующую выборки
    X_train, X_test, y_train, y_test = train_test_split(vectors, balanced_y, test_size=0.15, random_state=42)
    
    print('Data prepared.')
    
    # Обучим классификатор
    classifier = LinearSVC()
    classifier.fit(X_train, y_train)
    
    print('Model fitted.')
    
    # Посмотрим на результаты 
    preds = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    
    print('Model accuracy {0:.2f}%'.format(accuracy * 100))

In [6]:
rate_reviews(reviews)

Data prepared.
Model fitted.
Model accuracy 42.48%


**Ссылка на репозиторий github**

https://github.com/Some11One/M.SMART